In [1]:
import requests, json, os, re, sys, time, pandas as pd

In [2]:
sys.path.append(os.path.abspath('..'))
from Scripts.toolbox import addTitles

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\jeanm\.cache\huggingface\token
Login successful


In [3]:
results_path = "../Output/DataLLM/llama"
os.makedirs(results_path, exist_ok=True)

In [4]:
# Configuration API LM Studio
API_URL = "http://localhost:1234/v1/chat/completions"
HEADERS = {"Content-Type": "application/json"}

In [5]:
def read_txt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

In [6]:
def clean_keywords(raw_keywords, max_words=3):

    print(f"Entrée brute : \n{raw_keywords}")

    # Format avec astérisques (*)
    if "*" in raw_keywords:
        cleaned_keywords = re.findall(r"\* ([^\*]+)", raw_keywords)
    # Format avec numéros (1., 2., etc.)
    elif re.search(r"\d+\.", raw_keywords):
        cleaned_keywords = re.findall(r"\d+\.\s*([^\d]+)", raw_keywords)
    # Format avec tirets (-)
    elif "-" in raw_keywords:
        cleaned_keywords = re.findall(r"-\s*([^\n]+)", raw_keywords)
    # Format avec tirets bas (_)
    elif "_" in raw_keywords:
        cleaned_keywords = raw_keywords.split("_")
    # Format avec virgules
    elif "," in raw_keywords:
        cleaned_keywords = raw_keywords.split(",")
    # Format avec points (séparation de phrases)
    elif "." in raw_keywords:
        cleaned_keywords = raw_keywords.split(".")
    # Format avec lignes séparées par des sauts de ligne (nouvelle ligne pour chaque mot-clé)
    elif "\n" in raw_keywords:
        cleaned_keywords = raw_keywords.split("\n")
    else:
        # Cas générique où tout est dans une seule ligne
        cleaned_keywords = [raw_keywords]

    # Nettoyer chaque mot-clé (mettre en minuscules, enlever les espaces, etc.)
    cleaned_keywords = [kw.strip().lower() for kw in cleaned_keywords]

    # Supprimer "keywords:" et autres termes inutiles
    cleaned_keywords = [kw.replace("keywords:", "").strip() for kw in cleaned_keywords]

    # Exclusion des termes indésirables
    exclusion_terms = ['sdg', 'sdgs', 'sustainable development', 'indicator', 'sdg indicator', 'keyword', 'keywords']
    filtered_keywords = [
        kw for kw in cleaned_keywords if not any(excluded in kw for excluded in exclusion_terms)
    ]

    # Filtrer les mots-clés selon le nombre de mots et les exclure s'ils contiennent des chiffres
    results = [
        kw for kw in filtered_keywords
        if 2 <= len(kw.split()) <= max_words and not re.search(r'\d+', kw)
    ]

    # Supprimer les doublons en utilisant un set (pour garantir l'unicité)
    results = list(dict.fromkeys(results))

    print(f"Mots-clés nettoyés : {results}")
    return results

In [7]:
def manage_keywords(odd_number, txt_file, keywords):
    cible = txt_file[30:32].upper()
    indicator = txt_file[27:35].upper()
    output_keywords = []
    for keyword in keywords:
        output_keywords.append((f"ODD{odd_number}", f"Cible {odd_number}-{cible}", f"Indicateur {indicator}", keyword))

    return output_keywords

In [8]:
def extract_keywords(odd_number, txt_file, model):
    document = read_txt_file(txt_file)
    content = f"""Please extract only the keywords that are explicitly mentioned in this document.
    The keywords should consist of 2 to 3 words and should be meaningful within the context of this document.

    Do not include any explanations, definitions, or irrelevant terms. Return only the keywords as a comma-separated list python.
    Document:
    {document}"""


    # Préparer la requête
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You are an expert in natural language processing. Your task is to extract only the most relevant keywords from the user's document."},
            {"role": "user", "content": content}],
        "max_tokens": 500,
        "temperature": 0.0,
        "top_p": 0.9,
    }

    # Lancer le test
    response = requests.post(API_URL, headers=HEADERS, data=json.dumps(payload))

    if response.status_code == 200:
        # Extraire et nettoyer les mots-clés
        keywords = response.json()["choices"][0]["message"]["content"]
        cleaned_keywords = clean_keywords(keywords)
        managed_keywords = manage_keywords(odd_number, txt_file, cleaned_keywords)
        return managed_keywords

    else:
        print("Error:", response.status_code, response.text)
        return {"Keywords": "Error"}

In [9]:
def processing(dossier, model):
    start_time = time.time()

    # Récupération du numéro du dossier ODD (derniers deux chiffres)
    odd_number = dossier[-2:]

    print(f"\nDébut de traitement de l'ODD {odd_number}...")

    # Liste des fichiers texte à traiter
    txt_files = [fichier for fichier in os.listdir(dossier)
                 if fichier.endswith(".txt") and "keywords" not in fichier]

    keywords_ODD = []
    for txt_file in txt_files:
        keywords = extract_keywords(odd_number, f"{dossier}/{txt_file}", model)
        keywords_ODD.extend(keywords)


    # Stockage des résultats en dataframe
    df = pd.DataFrame(keywords_ODD, columns=["ODD", "Cible", "Indicateur", "Mots-cles"])

    # Ajout des titres des ODD, Cibles et Indicateurs
    df = addTitles(df)

    # Afficher les résultats
    display(df)

    # Exporter les données en csv
    df.to_csv(f"{results_path}/ODD{odd_number}_keywords.csv", index=False, sep=';')

    end_time = time.time()
    duree = end_time - start_time
    print(f"...Fin de traitement de l'ODD {odd_number}\nDurée d'exécution : {duree:.4f} secondes.\n")

    return None

In [10]:
model="llama-2-7b-chat"
# processing("../MetaData/ODD11", model)

In [11]:
if __name__ == "__main__":
    # Exécution de tous les ODD
    path = "../MetaData"
    odd_files = os.listdir(path)
    for odd_file in odd_files:
        processing(f"{path}/{odd_file}", model)
        print("----------------------------------------------------------------------------------------------------------")


Début de traitement de l'ODD 01...
Entrée brute : 
Keywords: poverty line, international poverty line, purchasing power, adjusted to local currency, $1 a day standard, $2.15, 2017 PPP, 2015 update, real value.
Mots-clés nettoyés : []
Entrée brute : 
Keywords: employment, poverty line, working poverty rate, employed population
Mots-clés nettoyés : ['poverty line', 'working poverty rate', 'employed population']
Entrée brute : 
Keywords: poverty rate, national poverty line, rural poverty rate, urban poverty rate, poverty line, purchasing power, cost of living, urban areas, rural areas.
Mots-clés nettoyés : ['poverty rate', 'national poverty line', 'rural poverty rate', 'urban poverty rate', 'poverty line', 'purchasing power', 'cost of living', 'urban areas', 'rural areas.']
Entrée brute : 
The following are the most relevant keywords extracted from the document:

* Poverty
* Multidimensional
* Headcount
* Deprivations
* Intensity
* Weighted
* Dimensions
* Indicators
* Methodology
* Alkir

,ODD,Cible,Indicateur,Mots-cles
0,ODD01 : End poverty in all its forms everywhere,"Cible 01-01 : By 2030, eradicate extreme pover...",Indicateur 01-01-01 : Proportion of the popula...,poverty line
1,ODD01 : End poverty in all its forms everywhere,"Cible 01-01 : By 2030, eradicate extreme pover...",Indicateur 01-01-01 : Proportion of the popula...,working poverty rate
2,ODD01 : End poverty in all its forms everywhere,"Cible 01-01 : By 2030, eradicate extreme pover...",Indicateur 01-01-01 : Proportion of the popula...,employed population
3,ODD01 : End poverty in all its forms everywhere,"Cible 01-02 : By 2030, reduce at least by half...",Indicateur 01-02-01 : Proportion of population...,poverty rate
4,ODD01 : End poverty in all its forms everywhere,"Cible 01-02 : By 2030, reduce at least by half...",Indicateur 01-02-01 : Proportion of population...,national poverty line
5,ODD01 : End poverty in all its forms everywhere,"Cible 01-02 : By 2030, reduce at least by half...",Indicateur 01-02-01 : Proportion of population...,rural poverty rate
6,ODD01 : End poverty in all its forms everywhere,"Cible 01-02 : By 2030, reduce at least by half...",Indicateur 01-02-01 : Proportion of population...,urban poverty rate
7,ODD01 : End poverty in all its forms everywhere,"Cible 01-02 : By 2030, reduce at least by half...",Indicateur 01-02-01 : Proportion of population...,poverty line
8,ODD01 : End poverty in all its forms everywhere,"Cible 01-02 : By 2030, reduce at least by half...",Indicateur 01-02-01 : Proportion of population...,purchasing power
9,ODD01 : End poverty in all its forms everywhere,"Cible 01-02 : By 2030, reduce at least by half...",Indicateur 01-02-01 : Proportion of population...,cost of living


...Fin de traitement de l'ODD 01
Durée d'exécution : 688.1290 secondes.

----------------------------------------------------------------------------------------------------------

Début de traitement de l'ODD 02...
Entrée brute : 
Keywords: undernourishment, prevalence, insufficient food intake, nutritional status, malnutrition, undernutrition, alimentation, alimentación, alimentazione
Mots-clés nettoyés : ['insufficient food intake', 'nutritional status']
Entrée brute : 
Food insecurity, hunger, measurement scale.
Mots-clés nettoyés : ['food insecurity', 'measurement scale.']
Entrée brute : 
Keywords: stunting, height-for-age, World Health Organization (WHO), Child Growth Standards, malnutrition, Joint Malnutrition Estimates (JME) working group, modelled estimates, primary data sources.
Mots-clés nettoyés : []
Entrée brute : 
Overweight, children, under 5 years.
Mots-clés nettoyés : []
Entrée brute : 
Keywords: wasting, children, under 5 years, weight-for-height, mid upper arm circum

,ODD,Cible,Indicateur,Mots-cles
0,"ODD02 : End hunger, achieve food security and ...","Cible 02-01 : By 2030, end hunger and ensure a...",Indicateur 02-01-01 : Prevalence of undernouri...,insufficient food intake
1,"ODD02 : End hunger, achieve food security and ...","Cible 02-01 : By 2030, end hunger and ensure a...",Indicateur 02-01-01 : Prevalence of undernouri...,nutritional status
2,"ODD02 : End hunger, achieve food security and ...","Cible 02-01 : By 2030, end hunger and ensure a...",Indicateur 02-01-02 : Prevalence of moderate o...,food insecurity
3,"ODD02 : End hunger, achieve food security and ...","Cible 02-01 : By 2030, end hunger and ensure a...",Indicateur 02-01-02 : Prevalence of moderate o...,measurement scale.
4,"ODD02 : End hunger, achieve food security and ...","Cible 02-02 : By 2030, end all forms of malnut...",Indicateur 02-02-03 : Prevalence of anaemia in...,iron deficiency
5,"ODD02 : End hunger, achieve food security and ...","Cible 02-02 : By 2030, end all forms of malnut...",Indicateur 02-02-03 : Prevalence of anaemia in...,iron deficiency anaemia
6,"ODD02 : End hunger, achieve food security and ...","Cible 02-05 : By 2020, maintain the genetic di...",Indicateur 02-05-01 : Number of (a) plant and ...,genetic resources
7,"ODD02 : End hunger, achieve food security and ...","Cible 02-05 : By 2020, maintain the genetic di...",Indicateur 02-05-01 : Number of (a) plant and ...,ex situ
8,"ODD02 : End hunger, achieve food security and ...","Cible 02-05 : By 2020, maintain the genetic di...",Indicateur 02-05-01 : Number of (a) plant and ...,in situ
9,"ODD02 : End hunger, achieve food security and ...","Cible 02-05 : By 2020, maintain the genetic di...",Indicateur 02-05-01 : Number of (a) plant and ...,breeding programmes


...Fin de traitement de l'ODD 02
Durée d'exécution : 569.8028 secondes.

----------------------------------------------------------------------------------------------------------

Début de traitement de l'ODD 03...
Entrée brute : 
Keywords: maternal mortality ratio, MMR, maternal death, pregnancy, live births, ICD, WHO
Mots-clés nettoyés : ['maternal mortality ratio', 'maternal death', 'live births']
Entrée brute : 
Keywords: skilled health personnel, competent MNH professionals, evidence-based care, human rights, quality care, socio-culturally sensitive care, dignified care, physiological processes, complications.
Mots-clés nettoyés : []
Entrée brute : 
The following are the relevant keywords extracted from the document:
under-five mortality rate, probability of death, life table, deaths per 1000 live births
Mots-clés nettoyés : []
Entrée brute : 
Keywords: mortality rate, neonatal, deaths, live births, early neonatal, late neonatal
Mots-clés nettoyés : ['mortality rate', 'live birth

...

...Fin de traitement de l'ODD 15
Durée d'exécution : 647.6442 secondes.

----------------------------------------------------------------------------------------------------------

Début de traitement de l'ODD 16...
Entrée brute : 
The following are the relevant keywords extracted from the document:
intentional homicide, unlawful death, population, ICCS 2015, ICD-10, ICD-11
Mots-clés nettoyés : []
Entrée brute : 
The following are the keywords extracted from the document:

* Conflict
* Deaths
* Armed conflict
* Population
* International Humanitarian Law (IHL)
* United Nations (UN)
* Direct deaths
* Indirect deaths
* Weapons
* Means and methods
* Cause of death

Please let me know if you need any further assistance.
Mots-clés nettoyés : ['armed conflict', 'united nations (un)', 'direct deaths', 'indirect deaths', 'means and methods']
Entrée brute : 
The following are the keywords extracted from the document:

* Physical violence
* Psychological violence
* Sexual violence
Mots-clés nett

,ODD,Cible,Indicateur,Mots-cles
0,ODD16 : Promote peaceful and inclusive societi...,Cible 16-01 : Significantly reduce all forms o...,Indicateur 16-01-02 : Conflict-related deaths ...,armed conflict
1,ODD16 : Promote peaceful and inclusive societi...,Cible 16-01 : Significantly reduce all forms o...,Indicateur 16-01-02 : Conflict-related deaths ...,united nations (un)
2,ODD16 : Promote peaceful and inclusive societi...,Cible 16-01 : Significantly reduce all forms o...,Indicateur 16-01-02 : Conflict-related deaths ...,direct deaths
3,ODD16 : Promote peaceful and inclusive societi...,Cible 16-01 : Significantly reduce all forms o...,Indicateur 16-01-02 : Conflict-related deaths ...,indirect deaths
4,ODD16 : Promote peaceful and inclusive societi...,Cible 16-01 : Significantly reduce all forms o...,Indicateur 16-01-02 : Conflict-related deaths ...,means and methods
...,...,...,...,...
76,ODD16 : Promote peaceful and inclusive societi...,Cible 16-10 : Ensure public access to informat...,Indicateur 16-10-02 : Number of countries that...,legal frameworks
77,ODD16 : Promote peaceful and inclusive societi...,Cible 16-10 : Ensure public access to informat...,Indicateur 16-10-02 : Number of countries that...,oversight mechanism
78,ODD16 : Promote peaceful and inclusive societi...,Cible 16-10 : Ensure public access to informat...,Indicateur 16-10-02 : Number of countries that...,appeals mechanism
79,ODD16 : Promote peaceful and inclusive societi...,Cible 16-10 : Ensure public access to informat...,Indicateur 16-10-02 : Number of countries that...,right to information


...Fin de traitement de l'ODD 16
Durée d'exécution : 1433.1433 secondes.

----------------------------------------------------------------------------------------------------------

Début de traitement de l'ODD 17...
Entrée brute : 
The following are the relevant keywords extracted from the document:

* Revenue
* Taxes
* Social contributions
* Grants
* Other revenue
* Property income
* Sales of goods and services
* Fines, penalties, and forfeits
* Transfers not elsewhere classified
* Premiums, fees, and claims related to non-life insurance and standardized guarantee schemes.
Mots-clés nettoyés : ['social contributions', 'other revenue', 'property income']
Entrée brute : 
Keywords extracted from the document:
budgetary central government, taxes, revenue, expense, ministry of finance, legislature, standards, classifications, international agreed definitions, unit of measurement.
Mots-clés nettoyés : ['ministry of finance', 'international agreed definitions', 'unit of measurement.']
Entré

,ODD,Cible,Indicateur,Mots-cles
0,ODD17 : Strengthen the means of implementation...,Cible 17-01 : Strengthen domestic resource mob...,Indicateur 17-01-01 : Total government revenue...,social contributions
1,ODD17 : Strengthen the means of implementation...,Cible 17-01 : Strengthen domestic resource mob...,Indicateur 17-01-01 : Total government revenue...,other revenue
2,ODD17 : Strengthen the means of implementation...,Cible 17-01 : Strengthen domestic resource mob...,Indicateur 17-01-01 : Total government revenue...,property income
3,ODD17 : Strengthen the means of implementation...,Cible 17-01 : Strengthen domestic resource mob...,Indicateur 17-01-02 : Proportion of domestic b...,ministry of finance
4,ODD17 : Strengthen the means of implementation...,Cible 17-01 : Strengthen domestic resource mob...,Indicateur 17-01-02 : Proportion of domestic b...,international agreed definitions
5,ODD17 : Strengthen the means of implementation...,Cible 17-01 : Strengthen domestic resource mob...,Indicateur 17-01-02 : Proportion of domestic b...,unit of measurement.
6,ODD17 : Strengthen the means of implementation...,Cible 17-02 : Developed countries to implement...,Indicateur 17-02-01 : Net official development...,development assistance committee
7,ODD17 : Strengthen the means of implementation...,Cible 17-03 : Mobilize additional financial re...,Indicateur 17-03-01 : Additional financial res...,foreign direct investment
8,ODD17 : Strengthen the means of implementation...,Cible 17-03 : Mobilize additional financial re...,Indicateur 17-03-01 : Additional financial res...,private grants
9,ODD17 : Strengthen the means of implementation...,Cible 17-03 : Mobilize additional financial re...,Indicateur 17-03-02 : Volume of remittances (i...,personal remittances


...Fin de traitement de l'ODD 17
Durée d'exécution : 1046.8745 secondes.

----------------------------------------------------------------------------------------------------------
